# Python Notebook for remote sensing work
Repository made for Artificial Intelligence & Neural Network course with Prof. Ciarfuglia @ Sapienza 

Import necessary items with this:

In [1]:
# Let's import our own modules
import sys
sys.path.insert(0, './src')
import utils
import dataset


import albumentations as A
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import warnings
import torch.utils.data as data
from rasterio.errors import NotGeoreferencedWarning
warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)

from rasterio.plot import show
import matplotlib.pyplot as plt

We should test that the CUDA platform has been successfully recognized and it's being in use with pyTorch

In [ ]:
device = utils.set_cuda_and_seed()

# TODO:
 - dataset analysis
 - mean and standard deviation calculatiom

# Data analysis

Before starting, we need to do some preliminary analysis on our dataset.
Inside split_dataset.py we already split our images into 3 different categories.
We'll be using:
 - train.txt as a training dataset
 - val.txt to validate that our model correctly predicts masks
 - test.txt to challenge different models one against the other

We should also calculate mean and std values of our train dataset, and we'll apply them to all of our sets


In [ ]:
# mean, std = utils.get_mean_std(path_to_train_data="data/train/AOI_11_Rotterdam/splits/train.txt")

# Create a dataset class

Now we'll need to create a SN6Dataset derived from data.Dataset that will import the data and return the data and labels, and that implements __len__ and __getitem__ methods
We'll be using RasterIO

We can see the implementation of the dataset in src/dataset.py

In [ ]:
transforms = A.Compose([
    A.Normalize(mean=[63.02, 66.64, 60.64], std=[55.5,55.35,52.63]),
    A.Resize(128, 128)
])

train_dataset = dataset.SN6Dataset('./data/train/AOI_11_Rotterdam', transform=transforms, split='train')
eval_dataset = dataset.SN6Dataset('./data/train/AOI_11_Rotterdam', transform=transforms, split='val')

# Use DataLoader

We'll now import the dataset into a dataloader, and just to see that everything's working we'll show the first image of the batch

Since we have normalized the images the image won't be shown correctly. That's normal, since now the values are centered in zero




In [ ]:
data_loader = data.DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = data.DataLoader(eval_dataset, batch_size=16, shuffle=True)

image, mask = next(iter(data_loader))
print(image[0].shape, mask[0].shape)

# Define the training model

We'll be using UNET, a convolutional neural network (CNN), for our task.

UNET is a popular architecture for image segmentation tasks. It is widely used in various domains, including medical imaging, remote sensing, and computer vision.


In [ ]:
from model import UNET
from train import train

model = UNET(in_channels = 3, out_channels = 1).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=optimizer.param_groups[0]['lr']*0.9)

train(data_loader, eval_loader, model, optimizer, criterion, scheduler, device, num_epochs=10)

Loss: 0.3261:   8%|▊         | 14/170 [00:10<02:05,  1.24it/s]

Output shape:  torch.Size([16, 128, 128])
Output type:  torch.float32
Mask shape:  torch.Size([16, 128, 128])
Mask type:  torch.float32


Loss: 0.2652:   9%|▉         | 15/170 [00:11<02:11,  1.18it/s]

Output shape:  torch.Size([16, 128, 128])
Output type:  torch.float32
Mask shape:  torch.Size([16, 128, 128])
Mask type:  torch.float32


Loss: 0.2526:   9%|▉         | 16/170 [00:12<02:20,  1.10it/s]

Output shape:  torch.Size([16, 128, 128])
Output type:  torch.float32
Mask shape:  torch.Size([16, 128, 128])
Mask type:  torch.float32


Loss: 0.1736:  10%|█         | 17/170 [00:13<02:21,  1.08it/s]

Output shape:  torch.Size([16, 128, 128])
Output type:  torch.float32
Mask shape:  torch.Size([16, 128, 128])
Mask type:  torch.float32
